# MODEL

## import libraries

In [16]:
import pandas as pd
import numpy as np
from wordcloud import WordCloud # for wordcloud
import matplotlib.pyplot as plt # for wordcloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

## data processing

In [9]:
test_data = pd.read_csv('6_data_country_and_label.csv')
# import a more balance train data
train_data = pd.read_csv('balance_data.csv') # find balance data from the first 600 data

test_x = test_data["summary"]
test_y = test_data["theme"]

train_x = train_data["summary"]
train_y = train_data["theme"]

categories = ["corporate and business topics", 
              "labor and employment issues", 
              "privacy, security, and cyber matters", 
              "legal and crime stories", 
              "government actions and regulations", 
              "technology and digital trends", 
              "environment and climate topics", 
              "social issues and activism", 
              "healthcare and medicine", 
              "community and cultural events", 
              "international relations and trade", 
              "education and learning", 
              "consumer topics", 
              "infrastructure and development", 
              "energy and resources", 
              "political topics and protests", 
              "media and communication", 
              "financial policies and taxation", 
              "human rights and social justice", 
              "science, research, and innovation", 
              "disaster and crisis management", 
              "organized crime and trafficking", 
              "sports, entertainment, and leisure", 
              "other", 
              "military"]

y_encoded = []
for each_theme in train_y:
    each_row = []
    for category in categories:
        if category in each_theme.lower():
            each_row.append(1)
        else:
            each_row.append(0)
    y_encoded.append(each_row)

# convert to dataframe
y_encoded = pd.DataFrame(y_encoded, columns = categories)
train_y = y_encoded

y_encoded = []
for each_theme in test_y:
    each_row = []
    for category in categories:
        if category in each_theme.lower():
            each_row.append(1)
        else:
            each_row.append(0)
    y_encoded.append(each_row)

y_encoded = pd.DataFrame(y_encoded, columns = categories)
test_y = y_encoded   

In [10]:
rows = np.where(np.sum(y_encoded, axis = 1) > 1)[0]
# print(rows)

theme_counts = y_encoded.to_numpy().argmax(axis = 1)
print(theme_counts)
unique_classes, counts = np.unique(theme_counts, 
                                   return_counts = True)
print(counts)
print(unique_classes)
print(len(unique_classes))


[ 1 17  2 ...  3  4 17]
[136  26  28 157  83 101  34  57 151 109 166  43  31  15  13  56  25  33
  16  49  16   7  85  36  35]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
25


## TFIDF (ngram range test accuracy to tune the preprocessing), countvectorizer (ngram range test accuracy to tune the preprocessing), word2vec, bert (not necessary)

### TDIDF

In [144]:
# word pattern
pattern = r"[a-zA-Z]+"

vectorizer = TfidfVectorizer(
    token_pattern = pattern, 
    stop_words = 'english', 
    ngram_range = (1, 2), # need to do the hyperparameter tuning for this later
    max_features = 100000 # can adjust to larger num too
)

vectorizer.fit(test_x)


# tokenization - create matrix
train_tokenized_features = vectorizer.transform(train_data["summary"])
test_tokenized_features = vectorizer.transform(test_data["summary"])


# create a dataframe
train_features = pd.DataFrame(
    data = train_tokenized_features.toarray(), 
    columns = vectorizer.get_feature_names_out()
)
test_features = pd.DataFrame(
    data = test_tokenized_features.toarray(), 
    columns = vectorizer.get_feature_names_out()
)


# print(len(vectorizer.get_feature_names_out()))

train_data["char_count"] = train_data["summary"].str.count(r"\S")
train_data["word_count"] = train_data["summary"].str.count(pattern)
train_data["avg_word_length"] = train_data["char_count"] / train_data["word_count"]

train_x = pd.concat([train_features, train_data.loc[:, "char_count": ]], axis = 1)
# data_y = data["theme"]

test_data["char_count"] = test_data["summary"].str.count(r"\S")
test_data["word_count"] = test_data["summary"].str.count(pattern)
test_data["avg_word_length"] = test_data["char_count"] / test_data["word_count"]

test_x = pd.concat([test_features, test_data.loc[:, "char_count": ]], axis = 1)


### Word2Vec - word embeddings

In [26]:
# tokenize all the sentences
tokenized_sentences = [word_tokenize(each_line[0].lower()) for each_line in train_data["summary"]]

# train word2vec model
word2vec_model = Word2Vec(
    sentences = tokenized_sentences, 
    vector_size = 1000, 
    window = 5, 
    min_count = 1, 
    workers = 4
)

# generate document vectors
def vectorize_doc(each_line):
    # remove out of vocab words
    words = [word for word in each_line if word in word2vec_model.wv]
    return np.mean(word2vec_model.wv[words], axis = 0) if words else np.zeros(word2vec_model.vector_size)

# create feature vectors 
train_x = np.array([vectorize_doc(word_tokenize(each_line.lower())) for each_line in train_data["summary"]])
test_x = np.array([vectorize_doc(word_tokenize(each_line.lower())) for each_line in test_data["summary"]])
# train_y = np.array([each_line.lower() for each_line in train_data["theme"]])

## logistic regression (softmax regression), support vector machine, bert, decision trees, random forest classifier, gradient boosting algorithm, KNN, neural networks (not necessary for small datasets), naive bayes classifier

### train test split

In [29]:
train_x, test_x, train_y, test_y = train_test_split(
    data_x, 
    y_encoded, 
    test_size = 0.6, 
    stratify = None, 
    random_state = 59
)

In [56]:
from sklearn.model_selection import StratifiedKFold

s = StratifiedKFold(n_splits = 2, 
                    shuffle = True, 
                    random_state = 59)

for train_i, test_i in s.split(data_x, y_encoded.to_numpy().argmax(axis = 1)):
    train_x, test_x = data_x.iloc[train_i], data_x.iloc[test_i]
    train_y, test_y = y_encoded.iloc[train_i], y_encoded.iloc[test_i]


print(len(train_x))
print(len(train_y))

c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(


754
754


### logistic regression one-vs-all OvA (softmax regression)

In [18]:
from sklearn.multiclass import OneVsRestClassifier

model = OneVsRestClassifier(LogisticRegression())
model.fit(train_x, train_y)

OneVsRestClassifier(estimator=LogisticRegression())

In [31]:
prediction = model.predict(test_x)
pred = (prediction > 0.001).astype(int)
pred_theme = [[categories[i] for i in range(len(categories)) if pred[j, i] == 1] for j in range(len(pred))]

print(pred_theme)

print("F1 Score: ", f1_score(test_y, pred, average = "micro"))

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[['corporate and business topics', 'labor and employment issues', 'privacy, security, and cyber matters', 'legal and crime stories', 'government actions and regulations', 'technology and digital trends', 'environment and climate topics', 'social issues and activism', 'healthcare and medicine', 'community and cultural events', 'international relations and trade', 'education and learning', 'consumer topics', 'infrastructure and development', 'energy and resources', 'political topics and protests', 'media and communication', 'financial policies and taxation', 'human rights and social justice', 'science, research, and innovation', 'disaster and crisis management', 'organized crime and trafficking', 'sports, entertainment, and leisure', 'other', 'military'], ['corporate and business topics', 'labor and employment issues', 'privacy, security, and cyber matters', 'legal and crime stories', 'government actions and regulations', 'technology and digital tre

### sigmoid activation function

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

#### train model

In [22]:
vocab_size = 100000
num_of_categories = len(categories)

model = Sequential([
    Embedding(input_dim = vocab_size, 
              output_dim = num_of_categories, 
              input_length = 1000
              ),
    GlobalAveragePooling1D(), 
    Dense(64, activation = "relu"), # hidden layer
    Dense(num_of_categories, activation = "sigmoid")
])

model.compile(optimizer = Adam(), 
              loss = "binary_crossentropy", 
              metrics = ["accuracy"])

model.summary()

c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

#### fit the model

In [23]:
model.fit(train_x, 
          train_y, 
          epochs = 10, 
          batch_size = 60, 
          validation_split = 0.3)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - accuracy: 0.0148 - loss: 0.6933 - val_accuracy: 0.2188 - val_loss: 0.6749
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0593 - loss: 0.6692 - val_accuracy: 0.2188 - val_loss: 0.6428
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.0593 - loss: 0.6321 - val_accuracy: 0.2188 - val_loss: 0.6087
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.0537 - loss: 0.5930 - val_accuracy: 0.2188 - val_loss: 0.5747
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0593 - loss: 0.5528 - val_accuracy: 0.0000e+00 - val_loss: 0.5395
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0296 - loss: 0.5098 - val_accuracy: 0.0000e+00 - val_loss: 0.5039
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.0296 - loss: 0.4661 - val_accuracy: 0.0000e+00 - val_loss: 0.4693
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0185 - loss: 0.4248 - val_accuracy: 0.0000e+00 - 

#### predict using the model (test_x)

In [97]:
print(test_x)

Index(['ability', 'ability business', 'absolutist', 'absolutist critics',
       'abuses', 'abuses including', 'access', 'access advanced',
       'access chinas', 'access fishing',
       ...
       'yuan', 'yuan billion', 'zealand', 'zealand ftas', 'zealand official',
       'zhang', 'zhang ruijin', 'char_count', 'word_count', 'avg_word_length'],
      dtype='object', length=3828)


In [24]:
prediction = model.predict(test_x)
pred = (prediction > 0.01).astype(int)
pred_theme = [[categories[i] for i in range(len(categories)) if pred[j, i] == 1] for j in range(len(pred))]

print(pred_theme)

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[['corporate and business topics', 'labor and employment issues', 'privacy, security, and cyber matters', 'legal and crime stories', 'government actions and regulations', 'technology and digital trends', 'environment and climate topics', 'social issues and activism', 'healthcare and medicine', 'community and cultural events', 'international relations and trade', 'education and learning', 'consumer topics', 'infrastructure and development', 'energy and resources', 'political topics and protests', 'media and communication', 'financial policies and taxation', 'human rights and social justice', 'science, research, and innovation', 'disaster and crisis management', 'organized crime and trafficking', 'sports, entertainment, and leisure', 'other', 'military'], ['corporate and business topics', 'labor and employment issues', 'privacy, security, and cyber matters', 'legal and crime stories', 'government actions and regulations', 'technology and digital tre

#### Evaluation of model

In [25]:
from sklearn.metrics import classification_report
print(classification_report(
    test_y, 
    pred, 
    target_names = categories
))

# Hamming Loss (the lower the better)
from sklearn.metrics import hamming_loss
hl = hamming_loss(test_y, pred)
print("Hamming Loss: ", hl)

# F1 score (the higher the better)
from sklearn.metrics import f1_score
f1 = f1_score(
    test_y, 
    pred, 
    average = "macro"
)
print("F1 Score: ", f1)

# Jaccard Score (the higher the better)
from sklearn.metrics import jaccard_score 
jaccard = jaccard_score(
    test_y, 
    pred, 
    average = "samples"
)
print("Jaccard Score: ", jaccard)


                                      precision    recall  f1-score   support

       corporate and business topics       0.09      1.00      0.16       134
         labor and employment issues       0.02      1.00      0.03        26
privacy, security, and cyber matters       0.02      1.00      0.04        28
             legal and crime stories       0.10      1.00      0.19       157
  government actions and regulations       0.06      1.00      0.11        86
       technology and digital trends       0.07      1.00      0.13       105
      environment and climate topics       0.02      1.00      0.05        35
          social issues and activism       0.04      1.00      0.08        60
             healthcare and medicine       0.10      1.00      0.18       151
       community and cultural events       0.07      1.00      0.13       109
   international relations and trade       0.12      1.00      0.21       176
              education and learning       0.03      1.00      

c:\Users\axlee\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Random forest classifier

In [ ]:
# binary relevance
# quite simple to understand, but it will ignore dependencies between labels 

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators = 100, 
                               random_state = 59)
model = MultiOutputClassifier(model)

model.fit(train_x, train_y)

pred = model.predict(test_x)

print("Accuracy: ", accuracy_score(test_y, pred))

Accuracy:  0.008620689655172414


In [40]:
# label powerset
# it can capture label dependencies but if too many label combinations
# it will lead to poor performance

from skmultilearn.problem_transform import LabelPowerset

model = LabelPowerset(RandomForestClassifier(n_estimators = 100, 
                                             random_state = 59))

model.fit(train_x, train_y)
pred = model.predict(test_x)

print("Accuracy: ", accuracy_score(test_y, pred))

Accuracy:  0.01989389920424403


### Support Vector Machine

In [ ]:
# one vs rest

from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

model = SVC(kernel = 'linear', # can change to 'rbf'
          probability = True)
model = MultiOutputClassifier(model)

model.fit(train_x, train_y)

pred = model.predict(test_x)
print("Accuracy: ", accuracy_score(test_y, pred))